In [1]:
import requests
from time import sleep, time
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup 
import pandas as pd
from multiprocessing import Pool

num_partitions = 20 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine
ROOT_URL = 'http://www.vanfun.net'

In [2]:
def get_listing_data(type_code, endpage):
    '''
    type_code 1 = house
    type_code 2 = townhouse
    type_code 3 = apartment
    '''
    all_page_response = []
    for page in tqdm(range(1,endpage)): 
        req = requests.get(ROOT_URL+'/house-{}-0-0-0-0-0-0-0-0-0-{}-0-0-0-0-0-0-0-0.aspx'.format(type_code,page))
        temp_soup = BeautifulSoup(req.text, "lxml")
        # 从第三个开始 前面两个是广告
        listing_div_list = temp_soup.find('dl',{"id":"goodsList"}).find_all("dd",title =True)[3:]
        all_page_response.append(listing_div_list)
        #sleep(1)
    return all_page_response


def get_info_per_page(listing_page):
    temp_page = pd.DataFrame()
    for house in listing_page:
        temp = pd.DataFrame({'address': house["title"],
                             'id': house.find_all('a')[0]['data-code'],
                             'house__detail_link':ROOT_URL+house.find_all('a')[0]['href'],
                             'house_config':house.find('span',{"class":"orange"}).text,
                             'list_comapny':house.find('span',{"class":"listingof"}).text,
                             'price_cad':house.find('span',{"class":"price"}).text,
                             'price_rmb':house.find('span',{"class":"price_rmb"}).text
                            }, index=[0])

        temp_page = pd.concat([temp_page, temp])
    return temp_page


def save_to_excel(type_code,endpage, reponses):

    total_listing = pd.DataFrame()
    chunk_of_df = []
    for page in tqdm(range(endpage-1)):
        chunk_of_df.append(get_info_per_page(reponses[page]))
    total_listing = pd.concat(chunk_of_df,ignore_index=True)
    total_listing.to_excel('total_vancouver_{}_data.xlsx'.format(type_code))
    
def get_house_detail_to_df(dataframe):
    # container
    ids = []
    temp_info = pd.DataFrame()
    
    for ide in tqdm(dataframe.id.values):
        req = requests.get(ROOT_URL+'/house-{}.aspx'.format(int(ide)))
        temp_soup = BeautifulSoup(req.text, "lxml")
        if temp_soup.find('tbody') is None:
            pass       
        else:
            # do stuff here
            house_info_table = temp_soup.find('tbody')
            # add record to dataframe
            house_info_table = pd.DataFrame({'MLS_number': house_info_table.find('input',{'class':'house_mslno'})['value'],
                             'listed_date': house_info_table.find('td',title=True).text,
                             'house_size':house_info_table.find_all('span',{'class':'numb area'})[0].text,
                             'land_size':house_info_table.find_all('span',{'class':'numb area'})[1].text ,
                             'feature':house_info_table.findAll('td',attrs={'class': None})[3].text,
                             'land_tax':house_info_table.findAll('td',attrs={'class': None})[5].text
                            }, index=[0])
            # add id
            ids.append(ide)
            temp_info = pd.concat([temp_info, house_info_table])
            sleep(1)
     # add ids to dataframe
    temp_info['id'] = ids
    temp_info = temp_info.reset_index(drop=True)     
    return temp_info


# def get_info_to_df(house_reponse, ids):
#     temp_info = pd.DataFrame()
#     for house in tqdm(house_reponse):
#         temp = pd.DataFrame({'MLS_number': house.find('input',{'class':'house_mslno'})['value'],
#                              'listed_date': house.find('td',title=True).text,
#                              'house_size':house.find_all('span',{'class':'numb area'})[0].text,
#                              'land_size':house.find_all('span',{'class':'numb area'})[1].text ,
#                              'feature':house.findAll('td',attrs={'class': None})[3].text,
#                              'land_tax':house.findAll('td',attrs={'class': None})[5].text
#                             }, index=[0])

#         temp_info = pd.concat([temp_info, temp])
#     temp_info['id'] = ids
#     temp_info = temp_info.reset_index(drop=True)
#     return temp_info


def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [3]:
house = pd.read_excel('vanfun_data/total_vancouver_1_data.xlsx')
town = pd.read_excel('vanfun_data/total_vancouver_2_data.xlsx')
apartment = pd.read_excel('vanfun_data/total_vancouver_3_data.xlsx')
total = pd.concat([house,town,apartment]).reset_index(drop=True)

In [9]:
# 
apartment_detail = parallelize_dataframe(apartment,get_house_detail_to_df)

100%|██████████| 270/270 [07:07<00:00,  1.55s/it]


In [19]:
apartment_detail = pd.read_excel('apartment_detail.xlsx')
townhouse = pd.read_excel('townhouse_detail.xlsx')
house = pd.read_excel('house_detail.xlsx')

In [21]:
total_detail = pd.concat([apartment_detail,townhouse,house]).reset_index(drop=True)

In [22]:
total = total.merge(total_detail,on='id').reset_index(drop=True)

In [24]:
total.to_excel('final.xlsx')